# References

# Modified from KEFE's preprocess_review.py

In [1]:
!python --version
!pwd

Python 3.6.3 :: Anaconda, Inc.
/gpfs/space/home/enlik/GitRepo/master-thesis-2021/notebooks


In [2]:
# from pl_utils import read_csv_from_gdrive, get_config, preprocess_review
from utils import *

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

config = get_config('config.yaml')

[nltk_data] Downloading package stopwords to
[nltk_data]     /gpfs/space/home/enlik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/gpfs/space/home/enlik/.local/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Let's Get Started

## Test

In [3]:
df = read_csv_from_gdrive(config['csv_input']['uber_google'])
total_reviews = len(df)
unique_users  = len(df['userName'].unique())
unknown_users = len(df[df['userName']=='A Google user'])
mean = df['score'].mean()

print(f'Total English reviews: {total_reviews} \n')
print(f'Total unique users : {unique_users}')
print(f'Total unknown users: {unknown_users}')
print(f'Total users who gave multiple reviews: {total_reviews - unique_users - unknown_users}\n')
print(f'Average rating for this app based on the textual reviews: {round(mean,2)} \n')

Total English reviews: 10000 

Total unique users : 9911
Total unknown users: 26
Total users who gave multiple reviews: 63

Average rating for this app based on the textual reviews: 3.3 



In [4]:
remove_things(df['content'])

0       amazing just absolutely love the cheap fares a...
1       superb services with uber cab  i loving it tha...
2       always on time  courteous drivers  highly reco...
3       i had to use lyft the last  days because uber ...
4       terrible  drivers are rude and uber don t even...
                              ...                        
9995           still the best experience around the globe
9996    the worst app which we are using today  instea...
9997    uber is a time waster compared to just phoning...
9998                           excellent service thus far
9999                            convinient and accessible
Name: content, Length: 10000, dtype: object